In [1]:
%matplotlib inline
import sys, os, re, csv, math, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D
from keras.layers import MaxPool1D, Flatten, Conv1D, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.layers import concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import initializers, regularizers, constraints, optimizers, layers

In [62]:
df['comment_text'][10]

'"\nFair use rationale for Image:Wonju.jpg\n\nThanks for uploading Image:Wonju.jpg. I notice the image page specifies that the image is being used under fair use but there is no explanation or rationale as to why its use in Wikipedia articles constitutes fair use. In addition to the boilerplate fair use template, you must also write out on the image description page a specific explanation or rationale for why using this image in each article is consistent with fair use.\n\nPlease go to the image description page and edit it to include a fair use rationale.\n\nIf you have uploaded other fair use media, consider checking that you have specified the fair use rationale on those pages too. You can find a list of \'image\' pages you have edited by clicking on the ""my contributions"" link (it is located at the very top of any Wikipedia page when you are logged in), and then selecting ""Image"" from the dropdown box. Note that any fair use images uploaded after 4 May, 2006, and lacking such a

In [24]:
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

In [25]:
url1 = 'https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_test.csv'
url2='https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_train.csv'
train = pd.read_csv(url2, encoding='utf-8')
test= pd.read_csv(url1, encoding='utf-8')
train.head(5)

,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...


In [26]:
test.head(5)

,Emotion,Text
0,sadness,I experienced this emotion when my grandfather...
1,neutral,"when I first moved in , I walked everywhere ...."
2,anger,"` Oh ! "" she bleated , her voice high and rath..."
3,fear,"However , does the right hon. Gentleman recogn..."
4,sadness,My boyfriend didn't turn up after promising th...


In [27]:
num_classes = 5
embed_num_dims = 300
max_seq_len = 500
class_names = ['joy', 'fear', 'anger', 'sadness', 'neutral']

In [28]:
X_train = train.Text.tolist()
X_test = test.Text.tolist()

y_train = train.Emotion
y_test = test.Emotion

data = train.append(test, ignore_index=True)

In [29]:
print(data.Emotion.value_counts())

joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


In [30]:
def clean_text(data):
    
    # remove hashtags and @usernames
    data = re.sub(r"(#[\d\w\.]+)", '', data)
    data = re.sub(r"(@[\d\w\.]+)", '', data)
    
    # tekenization using nltk
    data = word_tokenize(data)
    
    return data

In [31]:
texts = [' '.join(clean_text(text)) for text in data.Text]

texts_train = [' '.join(clean_text(text)) for text in X_train]
texts_test = [' '.join(clean_text(text)) for text in X_test]

In [32]:
print(texts_train[92])

a bit ? I 'm extremely annoyed that he did n't phone me when he promised me that he would ! He 's such a liar .


In [33]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

Number of unique words: 12087


In [34]:
X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len )

X_train_pad

array([[    0,     0,     0, ...,   119,    51,   345],
       [    0,     0,     0, ...,    37,   277,   154],
       [    0,     0,     0, ...,    16,     2,  1210],
       ...,
       [    0,     0,     0, ...,   876,     4,   909],
       [    0,     0,     0, ...,     1,     6,   117],
       [    0,     0,     0, ..., 10258,   173,    13]])

In [36]:
encoding = {
    'joy': 0,
    'fear': 1,
    'anger': 2,
    'sadness': 3,
    'neutral': 4
}
y_train = [encoding[x] for x in train.Emotion]
y_test = [encoding[x] for x in test.Emotion]

In [37]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [43]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open(filepath,encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix

In [41]:
import urllib.request
import zipfile
import os

fname = 'embeddings/wiki-news-300d-1M.vec'

if not os.path.isfile(fname):
    print('Downloading word vectors...')
    urllib.request.urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip',
                              'wiki-news-300d-1M.vec.zip')
    print('Unzipping...')
    with zipfile.ZipFile('wiki-news-300d-1M.vec.zip', 'r') as zip_ref:
        zip_ref.extractall('embeddings')
    print('done.')
    
    os.remove('wiki-news-300d-1M.vec.zip')

Unzipping...
done.


In [44]:
embedd_matrix = create_embedding_matrix(fname, index_of_words, embed_num_dims)
embedd_matrix.shape

(12088, 300)

In [45]:
new_words = 0
for word in index_of_words:
    entry = embedd_matrix[index_of_words[word]]
    if all(v == 0 for v in entry):
        new_words = new_words + 1

print('Words found in wiki vocab: ' + str(len(index_of_words) - new_words))
print('New words found: ' + str(new_words))

Words found in wiki vocab: 11442
New words found: 645


In [46]:
embedd_layer = Embedding(vocab_size,
                         embed_num_dims,
                         input_length = max_seq_len,
                         weights = [embedd_matrix],
                         trainable=False)

In [47]:
kernel_size = 3
filters = 256

model = Sequential()
model.add(embedd_layer)
model.add(Conv1D(filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [48]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 300)          3626400   
                                                                 
 conv1d (Conv1D)             (None, 498, 256)          230656    
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 3,924,133
Trainable params: 297,733
Non-trainable params: 3,626,400
________________________________________

In [51]:
batch_size = 256
epochs = 6

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00000001)

callbacks = [learning_rate_reduction, 
             EarlyStopping('val_loss', patience=3), 
             ModelCheckpoint('comment_sentiment_type_model.h5', save_best_only=True)]

hist = model.fit(X_train_pad, y_train, 
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(X_test_pad,y_test),callbacks=callbacks)

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.__init__.<locals>.permutation at 0x0000018255CD15E0>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=int64>,)
    kwargs: {}

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.__init__.<locals>.slice_batch_indices at 0x0000018255CD1A60>
    args: (<tf.Tensor 'args_0:0' shape=(7934,) dtype=int64>,)
    kwargs: {}

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.slice_inputs.<locals>.grab_batch at 0x0000018255CD1790>
    args: (<tf.Tensor 'args_0:0' shape=(None,) dtype=int64>, (<tf.Tensor 'args_1:0' shape=(7934, 500) dtype=int32>, <tf.Tensor 'args_2:0' shape=(7934, 5) dtype=float32>))
    kwargs: {}

Epoch 1/6
31/31 [==============================] - 16s 520ms/step - loss: 0.5922 - accuracy: 0.7969 - val_loss: 0.7272 - val_accuracy: 0.7395 - lr: 0.0010
Epoch 2/6
31/31 [==============================] - 17s 548ms/step - loss: 0.4780 - accuracy: 0.8479 - val_loss: 0.7181 - val_accuracy: 0.7501 - l

In [53]:
predictions = model.predict(X_test_pad)
predictions = np.argmax(predictions, axis=1)
predictions = [class_names[pred] for pred in predictions]

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.__init__.<locals>.permutation at 0x0000018255CD1670>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=int64>,)
    kwargs: {}

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.__init__.<locals>.slice_batch_indices at 0x0000018255CD13A0>
    args: (<tf.Tensor 'args_0:0' shape=(3393,) dtype=int64>,)
    kwargs: {}

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.slice_inputs.<locals>.grab_batch at 0x0000018255D07160>
    args: (<tf.Tensor 'args_0:0' shape=(None,) dtype=int64>, <tf.Tensor 'args_1:0' shape=(3393, 500) dtype=int32>)
    kwargs: {}

INFO:tensorflow:Converted call: <function Model.make_predict_function.<locals>.predict_function at 0x0000018255D074C0>
    args: (<tensorflow.python.data.ops.iterator_ops.OwnedIterator object at 0x0000018255D1A820>,)
    kwargs: {}

INFO:tensorflow:<function Model.make_predict_function.<locals>.predict_function at 0x0000018255D074C0> is not cached for su

In [55]:
print("Accuracy: {:.2f}%".format(accuracy_score(test.Emotion, predictions) * 100))
print("\nF1 Score: {:.2f}".format(f1_score(test.Emotion, predictions, average='micro') * 100))

Accuracy: 76.48%

F1 Score: 76.48


In [66]:
import time

message = ['''I love you''']

seq = tokenizer.texts_to_sequences(message)
padded = pad_sequences(seq, maxlen=max_seq_len)

start_time = time.time()
pred = model.predict(padded)

print('Message: ' + str(message))
print('🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🤖🤖🤖🤖🤖🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠')
print('predicted: {} ({:.2f} seconds)'.format(class_names[np.argmax(pred)], (time.time() - start_time)))

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.__init__.<locals>.permutation at 0x0000018263F96160>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=int64>,)
    kwargs: {}

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.__init__.<locals>.slice_batch_indices at 0x0000018255D50430>
    args: (<tf.Tensor 'args_0:0' shape=(1,) dtype=int64>,)
    kwargs: {}

INFO:tensorflow:Converted call: <function TensorLikeDataAdapter.slice_inputs.<locals>.grab_batch at 0x000001826BC8BC10>
    args: (<tf.Tensor 'args_0:0' shape=(None,) dtype=int64>, <tf.Tensor 'args_1:0' shape=(1, 500) dtype=int32>)
    kwargs: {}

Message: ['I love you']
🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🤖🤖🤖🤖🤖🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠
predicted: neutral (0.06 seconds)


In [1]:
from facebook_scraper import get_posts

In [5]:
listposts = []
for post in get_posts("MetaIndia", pages=1):#366190054572553
    print(post['text'][:100])
    listposts.append(post)

We’re proud to continue our collaboration with MyGov to offer Digilocker services on WhatsApp for ef
At Meta, we work hard to ensure our users are safe online. One of the ways they can do so is to let 


In [24]:
%%writefile FB_post_sentiment.py
import streamlit as st
import filetype
import numpy as np
from io import StringIO
import pandas as pd
import sys, os, re, csv, math, codecs, numpy as np, pandas as pd
import time
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D
from keras.layers import MaxPool1D, Flatten, Conv1D, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.layers import concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import initializers, regularizers, constraints, optimizers, layers
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import tweepy
from tweepy import OAuthHandler
from facebook_scraper import get_posts
tokenizer = Tokenizer()

from keras.models import load_model
model = load_model('comment_sentiment_type_model.h5')
class_names = ['joy', 'fear', 'anger', 'sadness', 'neutral']
count=10
input_ = st.text_input('Input username', '#India')
st.write('Your input', input_)
if st.button('Predict_sentiment'):
    listposts = []
    for post in get_posts("MetaIndia", pages=1):#366190054572553
        txt=post['text'][:100]
        listposts.append(post)
        seq = tokenizer.texts_to_sequences(txt)
        padded = pad_sequences(seq, maxlen=500)

        start_time = time.time()
        pred = model.predict(padded)
        st.write('Message: ' + str(txt))
        st.write('🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🤖🤖🤖🤖🤖🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠')
        st.write('Predicted: {} ({:.2f} seconds)'.format(class_names[np.argmax(pred)], (time.time() - start_time)))

Writing FB_post_sentiment.py


In [25]:
#Twitter MetaIndia

In [26]:
!streamlit run FB_post_sentiment.py

^C
